In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)

%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


In [6]:
df["target_enc_content_id"] = df.groupby("content_id")["answered_correctly"].transform(f)

In [9]:
df["rate"] = df["answered_correctly"] - df["target_enc_content_id"]
group = df.groupby("user_id")
for i in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 100, 300, 500, 1000]:
    for col in ["content_id", "part"]:
        col_name = f"{col}_shift{i}"
        df[col_name] = group[col].shift(i).fillna(-1)
        df["is_same"] = df[col] == df[f"{col}_shift{i}"]
        print("=================")
        print(f"col={col} shift({i})")
        print(df.groupby("is_same")["answered_correctly"].describe())

col=content_id shift(1)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    9814392.0  0.655026  0.475360  0.0  0.0  1.0  1.0  1.0
True       23150.0  0.651663  0.476454  0.0  0.0  1.0  1.0  1.0
col=part shift(1)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False     737086.0  0.637919  0.480602  0.0  0.0  1.0  1.0  1.0
True     9100456.0  0.656403  0.474908  0.0  0.0  1.0  1.0  1.0
col=content_id shift(2)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    9797190.0  0.654234  0.475617  0.0  0.0  1.0  1.0  1.0
True       40352.0  0.845534  0.361399  0.0  1.0  1.0  1.0  1.0
col=part shift(2)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    1333735.0  

col=content_id shift(15)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    9831194.0  0.654891  0.475404  0.0  0.0  1.0  1.0  1.0
True        6348.0  0.852552  0.354580  0.0  1.0  1.0  1.0  1.0
col=part shift(15)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    5432107.0  0.643814  0.478871  0.0  0.0  1.0  1.0  1.0
True     4405435.0  0.668834  0.470633  0.0  0.0  1.0  1.0  1.0
col=content_id shift(100)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    9834955.0  0.654991  0.475371  0.0  0.0  1.0  1.0  1.0
True        2587.0  0.760727  0.426722  0.0  1.0  1.0  1.0  1.0
col=part shift(100)
             count      mean       std  min  25%  50%  75%  max
is_same                                                        
False    70683

# 連続で同じ問題を解く人

In [ ]:
df["is_same"] = df["content_id"] == df["content_id_shift1"]

In [17]:
df[df["is_same"] == 1].head(5)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tag,type_of,target_enc_content_id,rate,content_id_shift1,is_same,part_shift1,content_id_shift2,part_shift2,content_id_shift3,part_shift3,content_id_shift4,part_shift4,content_id_shift5,part_shift5,content_id_shift6,part_shift6,content_id_shift7,part_shift7,content_id_shift8,part_shift8,content_id_shift9,part_shift9,content_id_shift10,part_shift10,content_id_shift11,part_shift11,content_id_shift12,part_shift12,content_id_shift13,part_shift13,content_id_shift14,part_shift14,content_id_shift15,part_shift15
2501,155960097,475020,834,0,220,0,0.0,22000.0,True,834,3,2,148 32 38 92,-1,NaN,0.333333,-0.333333,834.0,True,2.0,1109.0,2.0,1118.0,2.0,12079.0,2.0,853.0,2.0,808.0,2.0,795.0,2.0,2493.0,3.0,2492.0,3.0,2491.0,3.0,2510.0,3.0,2511.0,3.0,2509.0,3.0,1449.0,3.0,1450.0,3.0
2735,414779688,475020,278,0,358,1,1.0,18000.0,True,278,1,2,2 107 81 92,-1,NaN,0.500000,0.500000,278.0,True,2.0,795.0,2.0,12208.0,2.0,1314.0,2.0,10045.0,6.0,10042.0,6.0,10044.0,6.0,10043.0,6.0,6915.0,6.0,6912.0,6.0,6914.0,6.0,6913.0,6.0,14811.0,5.0,5673.0,5.0,6668.0,5.0
3193,1125440,496840,5658,0,10,3,1.0,39000.0,False,5658,3,5,26,-1,NaN,1.000000,0.000000,5658.0,True,5.0,4076.0,5.0,3986.0,5.0,6199.0,5.0,9315.0,5.0,4853.0,5.0,296.0,2.0,5358.0,5.0,5418.0,5.0,4505.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3253,19925381,496840,258,0,73,1,1.0,17000.0,True,258,1,2,143 30 81 92,-1,NaN,0.333333,0.666667,258.0,True,2.0,29544.0,2.0,405.0,2.0,294.0,2.0,1315.0,2.0,951.0,2.0,1291.0,2.0,10687.0,2.0,834.0,2.0,773.0,2.0,738.0,2.0,10686.0,2.0,1260.0,2.0,638.0,2.0,853.0,2.0
3262,20333859,496840,855,0,81,0,0.0,14000.0,True,855,1,2,2 107 62 38 102,-1,NaN,1.000000,-1.000000,855.0,True,2.0,553.0,2.0,446.0,2.0,403.0,2.0,1265.0,2.0,695.0,2.0,255.0,2.0,738.0,2.0,258.0,2.0,258.0,2.0,29544.0,2.0,405.0,2.0,294.0,2.0,1315.0,2.0,951.0,2.0


# user_id == 475020

In [25]:
df[df["user_id"] == 475020].to_csv("020_previous_xxx/475020.csv", index=False)
df[df["user_id"] == 496840].to_csv("020_previous_xxx/496840.csv", index=False)

# 1個とばし

In [27]:
df["is_same"] = df["content_id"] == df["content_id_shift2"]
df[df["is_same"] == 1].head(30)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tag,type_of,target_enc_content_id,rate,content_id_shift1,is_same,part_shift1,content_id_shift2,part_shift2,content_id_shift3,part_shift3,content_id_shift4,part_shift4,content_id_shift5,part_shift5,content_id_shift6,part_shift6,content_id_shift7,part_shift7,content_id_shift8,part_shift8,content_id_shift9,part_shift9,content_id_shift10,part_shift10,content_id_shift11,part_shift11,content_id_shift12,part_shift12,content_id_shift13,part_shift13,content_id_shift14,part_shift14,content_id_shift15,part_shift15
1769,11930252533,408250,3628,0,119,2,1.0,27000.0,True,3628,2,5,108,-1,NaN,0.333333,0.666667,27845.0,True,5.0,3628.0,5.0,5582.0,5.0,8688.0,5.0,4112.0,5.0,3955.0,5.0,3255.0,5.0,4112.0,5.0,6422.0,5.0,6070.0,5.0,1814.0,3.0,1815.0,3.0,1816.0,3.0,1414.0,3.0,1413.0,3.0
1791,12487799023,408250,4556,0,142,2,1.0,93000.0,True,4556,2,5,182,-1,NaN,0.000000,1.000000,17914.0,True,5.0,4556.0,5.0,4806.0,5.0,9804.0,5.0,5274.0,5.0,4131.0,5.0,4501.0,5.0,8004.0,5.0,5339.0,5.0,4435.0,5.0,5264.0,5.0,8656.0,5.0,5828.0,5.0,3603.0,5.0,4762.0,5.0
1810,12488775913,408250,5334,0,162,2,1.0,31000.0,True,5334,2,5,44,-1,NaN,0.000000,1.000000,24985.0,True,5.0,5334.0,5.0,5945.0,5.0,6661.0,5.0,8777.0,5.0,3873.0,5.0,5717.0,5.0,4067.0,5.0,8953.0,5.0,5835.0,5.0,4114.0,5.0,3866.0,5.0,9198.0,5.0,5154.0,5.0,5368.0,5.0
2105,18305358587,408250,5400,0,398,1,1.0,16000.0,True,5400,1,5,133,-1,NaN,0.000000,1.000000,12811.0,True,5.0,5400.0,5.0,6154.0,5.0,4976.0,5.0,3916.0,5.0,9259.0,5.0,3609.0,5.0,5691.0,5.0,8567.0,5.0,5164.0,5.0,10575.0,1.0,7903.0,1.0,7972.0,1.0,7871.0,1.0,10621.0,1.0
2109,18305512291,408250,4434,0,403,3,1.0,33000.0,True,4434,3,5,26,-1,NaN,0.500000,0.500000,15888.0,True,5.0,4434.0,5.0,8696.0,5.0,5367.0,5.0,5400.0,5.0,12811.0,5.0,5400.0,5.0,6154.0,5.0,4976.0,5.0,3916.0,5.0,9259.0,5.0,3609.0,5.0,5691.0,5.0,8567.0,5.0,5164.0,5.0
2285,83580352,475020,6167,0,99,1,0.0,29000.0,True,6167,2,5,175,-1,NaN,0.000000,0.000000,28621.0,True,5.0,6167.0,5.0,13411.0,5.0,3940.0,5.0,4026.0,5.0,11717.0,5.0,13418.0,5.0,9474.0,5.0,8706.0,5.0,5156.0,5.0,13317.0,5.0,5332.0,5.0,9496.0,5.0,4453.0,5.0,13368.0,5.0
2827,528172405,475020,5348,0,407,3,1.0,51000.0,True,5348,3,5,179,-1,NaN,0.000000,1.000000,484.0,True,5.0,5348.0,5.0,13404.0,5.0,9476.0,5.0,11287.0,5.0,6180.0,5.0,10271.0,6.0,10270.0,6.0,10269.0,6.0,10268.0,6.0,10018.0,6.0,10019.0,6.0,10020.0,6.0,10017.0,6.0,24558.0,6.0
3409,1557068604,496840,6046,0,211,1,0.0,35000.0,True,6046,3,5,147,-1,NaN,1.000000,-1.000000,5596.0,True,5.0,6046.0,5.0,15858.0,5.0,4423.0,5.0,8428.0,5.0,5173.0,5.0,5596.0,5.0,4065.0,5.0,11504.0,5.0,3749.0,5.0,5427.0,5.0,5350.0,5.0,11403.0,5.0,4451.0,5.0,6143.0,5.0
4036,120712125,583550,6649,0,165,2,1.0,30000.0,True,6649,2,5,60,-1,NaN,0.000000,1.000000,3255.0,True,5.0,6649.0,5.0,3894.0,5.0,5551.0,5.0,26516.0,5.0,3894.0,5.0,5628.0,5.0,8614.0,5.0,3854.0,5.0,5285.0,5.0,4250.0,5.0,9425.0,5.0,2958.0,4.0,2959.0,4.0,2960.0,4.0
4226,1101202,999270,4587,0,18,1,1.0,35000.0,True,4587,1,5,79,-1,NaN,0.000000,1.000000,27022.0,True,5.0,4587.0,5.0,9495.0,5.0,5674.0,5.0,6193.0,5.0,6261.0,5.0,8602.0,5.0,5339.0,5.0,5912.0,5.0,3980.0,5.0,8225.0,5.0,4074.0,5.0,6071.0,5.0,5987.0,5.0,5689.0,5.0


In [28]:
df[df["user_id"] == 408250].to_csv("020_previous_xxx/408250.csv", index=False)
df[df["user_id"] == 1283420].to_csv("020_previous_xxx/1283420.csv", index=False)

# 2個飛ばし

In [29]:
df["is_same"] = df["content_id"] == df["content_id_shift3"]
df[df["is_same"] == 1].head(30)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tag,type_of,target_enc_content_id,rate,content_id_shift1,is_same,part_shift1,content_id_shift2,part_shift2,content_id_shift3,part_shift3,content_id_shift4,part_shift4,content_id_shift5,part_shift5,content_id_shift6,part_shift6,content_id_shift7,part_shift7,content_id_shift8,part_shift8,content_id_shift9,part_shift9,content_id_shift10,part_shift10,content_id_shift11,part_shift11,content_id_shift12,part_shift12,content_id_shift13,part_shift13,content_id_shift14,part_shift14,content_id_shift15,part_shift15
1765,11929640651,408250,4112,0,114,3,1.0,20000.0,True,4112,3,5,60,-1,NaN,0.000000,1.000000,3955.0,True,5.0,3255.0,5.0,4112.0,5.0,6422.0,5.0,6070.0,5.0,1814.0,3.0,1815.0,3.0,1816.0,3.0,1414.0,3.0,1413.0,3.0,1412.0,3.0,1528.0,3.0,1527.0,3.0,1526.0,3.0,2093.0,3.0
1815,12489397834,408250,4490,0,168,2,1.0,26000.0,True,4490,2,5,80,-1,NaN,0.000000,1.000000,5238.0,True,5.0,7949.0,5.0,4490.0,5.0,8829.0,5.0,6108.0,5.0,5334.0,5.0,24985.0,5.0,5334.0,5.0,5945.0,5.0,6661.0,5.0,8777.0,5.0,3873.0,5.0,5717.0,5.0,4067.0,5.0,8953.0,5.0
1853,12663261320,408250,3642,0,207,2,1.0,11000.0,True,3642,2,5,49,-1,NaN,0.333333,0.666667,4141.0,True,5.0,8330.0,5.0,3642.0,5.0,9152.0,5.0,5270.0,5.0,5197.0,5.0,3999.0,5.0,9010.0,5.0,5439.0,5.0,6371.0,5.0,4108.0,5.0,8905.0,5.0,5648.0,5.0,4228.0,5.0,6057.0,5.0
3968,73296673,583550,5670,0,101,3,1.0,10000.0,True,5670,3,5,123,-1,NaN,0.666667,0.333333,4120.0,True,5.0,6481.0,5.0,5670.0,5.0,3731.0,5.0,3635.0,5.0,8307.0,5.0,4117.0,5.0,4975.0,5.0,6312.0,5.0,8947.0,5.0,6016.0,5.0,4120.0,5.0,1758.0,3.0,1759.0,3.0,1757.0,3.0
4034,120417547,583550,3894,0,162,1,1.0,30000.0,True,3894,1,5,1,-1,NaN,0.000000,1.000000,5551.0,True,5.0,26516.0,5.0,3894.0,5.0,5628.0,5.0,8614.0,5.0,3854.0,5.0,5285.0,5.0,4250.0,5.0,9425.0,5.0,2958.0,4.0,2959.0,4.0,2960.0,4.0,3423.0,4.0,3424.0,4.0,3425.0,4.0
4330,136834852,1173490,1315,0,67,3,1.0,13000.0,True,1315,3,2,2 107 62 29 102,-1,NaN,0.571429,0.428571,824.0,True,2.0,824.0,2.0,1315.0,2.0,553.0,2.0,3153.0,2.0,553.0,2.0,1131.0,2.0,1265.0,2.0,876.0,2.0,242.0,2.0,10684.0,2.0,335.0,2.0,651.0,2.0,421.0,2.0,10686.0,2.0
4406,339024920,1173490,3965,0,145,2,1.0,26000.0,True,3965,2,5,60,-1,NaN,0.666667,0.333333,5394.0,True,5.0,3255.0,5.0,3965.0,5.0,6393.0,5.0,9187.0,5.0,4424.0,5.0,4065.0,5.0,6460.0,5.0,279.0,2.0,675.0,2.0,447.0,2.0,1313.0,2.0,1202.0,2.0,258.0,2.0,329.0,2.0
4422,364771140,1173490,853,0,161,1,0.0,15000.0,True,853,0,2,129 143 30 38 102,-1,NaN,0.500000,-0.500000,258.0,True,2.0,29544.0,2.0,853.0,2.0,1036.0,2.0,507.0,2.0,1002.0,2.0,920.0,2.0,1260.0,2.0,1194.0,2.0,1010.0,2.0,1367.0,2.0,638.0,2.0,5588.0,5.0,4851.0,5.0,8438.0,5.0
4431,400047187,1173490,5836,0,173,3,1.0,23000.0,True,5836,3,5,14,-1,NaN,0.333333,0.666667,4380.0,True,5.0,14323.0,5.0,5836.0,5.0,3986.0,5.0,258.0,2.0,294.0,2.0,683.0,2.0,538.0,2.0,6440.0,5.0,853.0,2.0,258.0,2.0,29544.0,2.0,853.0,2.0,1036.0,2.0,507.0,2.0
4533,1226676505,1173490,5900,0,279,0,1.0,21000.0,True,5900,0,5,89,-1,NaN,0.000000,1.000000,4851.0,True,5.0,3123.0,5.0,5900.0,5.0,6173.0,5.0,6453.0,5.0,8369.0,5.0,4439.0,5.0,6671.0,5.0,1334.0,2.0,981.0,2.0,244.0,2.0,536.0,2.0,731.0,2.0,904.0,2.0,589.0,2.0
